In [114]:
import geopandas as gpd

# Load already updated shapefile
gdf = gpd.read_file('C:/WORK_IAAC/submissions/Internet of Buildings/IoB_WIP/IoB_QGIS/buildings/buildings_FIN.shp', encoding='latin-1')
gdf.tail()

,osm_id,addr_stree,addr_house,building_l,building,name,build_prog,geometry
113137,1.158969e+09,NaN,NaN,NaN,yes,noname,unknown,"POLYGON ((103.78102 1.40789, 103.78103 1.40803..."
113138,1.158969e+09,NaN,NaN,NaN,yes,noname,unknown,"POLYGON ((103.78086 1.40790, 103.78090 1.40803..."
113139,1.158969e+09,NaN,NaN,NaN,yes,noname,unknown,"POLYGON ((103.78068 1.40798, 103.78071 1.40806..."
113140,1.158969e+09,NaN,NaN,NaN,yes,noname,unknown,"POLYGON ((103.78046 1.40785, 103.78061 1.40798..."
113141,1.158969e+09,NaN,NaN,1,yes,noname,unknown,"POLYGON ((103.77981 1.40798, 103.77984 1.40807..."


In [59]:
gdf['building'].unique()

array(['train_station', 'civic', 'transportation', 'retail', 'public',
       'commercial', 'yes', 'office', 'college', 'industrial', 'market',
       'government', 'hospital', 'apartments', 'residential', 'hotel',
       'house', 'garage', 'temple', 'university', 'residential_senior',
       'social_club', 'warehouse', 'service', 'school', 'socialclub',
       'grandstand', 'greenhouse', 'medical', 'roof', 'parking',
       'sports_centre', 'mosque', 'scientific', 'hangar', 'church',
       'shop', 'cultural', 'hut', 'columbarium', 'construction', 'cafe',
       'manufacture', 'boathouse', 'dormitory', 'terrace',
       'semidetached_house', 'toilets', 'ruins', 'kindergarten',
       'detached', 'swimming_pool_changing_room', 'fastfood',
       'fire_station', 'religious', 'parlour', 'mix_used', 'education',
       'recreational', 'shed', 'stable', 'gazebo', 'shrine',
       'CET_Campus_East', 'wayside_shrine', 'EiS_Residences', 'stadium',
       'farm_auxiliary', 'bungalow', 'chapel'

In [107]:
gdf.head()

,fid,osm_id,addr_stree,addr_house,building_l,building,name,addr_city,office,geometry,build_program
0,1.0,22875450.0,NaN,NaN,NaN,train_station,noname,NaN,NaN,"POLYGON ((103.74901 1.34852, 103.74912 1.34866...",transport
1,2.0,22886442.0,Boon Lay Way,301,NaN,train_station,noname,Singapore,NaN,"POLYGON ((103.70508 1.33839, 103.70518 1.33842...",transport
2,3.0,24680746.0,NaN,NaN,NaN,civic,noname,NaN,NaN,"POLYGON ((103.88773 1.37072, 103.88788 1.37084...",civic
3,4.0,26561741.0,Airport Boulevard,80,3,transportation,Changi Airport Terminal 1,Singapore,NaN,"POLYGON ((103.98765 1.36006, 103.98854 1.36213...",transport
4,5.0,30527021.0,Rochor Canal Road,1,6,retail,Sim Lim Square,Singapore,NaN,"POLYGON ((103.85254 1.30299, 103.85267 1.30316...",deco_box


In [109]:
gdf['addr_city'].unique()

array([nan, 'Singapore', 'Pulau Ubin, Singapore', 'Sembawang',
       'Ang Mo Kio', 'singapore', 'Woodlands', 'Woodlands Spectrum II',
       'Yishun', 'SG', '<different>', '563455', 'Singapoer', 'SINGAPORE',
       'SIngapore'], dtype=object)

In [51]:
#check if column values are NaN
gdf['source'].isna().sum()

90326

In [52]:
#count total number of no-value rows in building column
gdf['building'].value_counts().get('yes', 0)

72396

In [53]:
#count total number of rows in building column
gdf.shape[0]

113142

In [57]:
#replace function type in column 'building' by mask from 'name' if 'building' has no value ('yes')
word = 'cafe'
mask = gdf['name'].str.contains('Cafe', case=False)
gdf.loc[mask & ((gdf['building'] == 'yes') | (gdf['building'] == 'no')), 'building'] = word
#check the difference before and after
gdf['building'].value_counts().get(word, 0)

12

In [61]:
#add column with current building program or function
#residential_highrise, residential_lowrise, civic, small_box, deco_box, education, glass_box, religious, sport, ruins, construction_site, transport, industrial, medic 
def map_values(x):
    if x in ['residential', 'apartments','EiS_Residences','mix_used','dormitory','hotel','residential_senior']:
        return 'resident_high'
    elif x in ['house', 'shelter', 'bungalow','hut','terrace','semidetached_house','detached','hall']:
        return 'resident_low'
    elif x in ['civic', 'public', 'social_club','socialclub','government','scientific','columbarium','fire_station','jtc_nanospace','tourism']:
        return 'civic'
    elif x in ['garage', 'service','roof','hangar','shop','cafe','toilets','fastfood','parlour','shed','gazebo','farm_auxiliary','guardhouse','security','seasonal','Security_Post','gatehouse','tent','shelter', 'utility']:
        return 'small_box'
    elif x in ['retail', 'commercial','market', 'warehouse','cultural','recreational','multi-purpose_stage','pavilion','supermarket','multi-purpose_hall']:
        return 'deco_box'
    elif x in ['college', 'university','school','kindergarten','education','CET_Campus_East']:
        return 'education'
    elif x in ['office', 'greenhouse']:
        return 'glass_box' 
    elif x in ['temple', 'mosque','church','religious','wayside_shrine','chapel','pagoda']:
        return 'religious' 
    elif x in ['grandstand', 'sports_centre','swimming_pool_changing_room','stable','stadium']:
        return 'sport'     
    elif x in ['train_station', 'transportation','parking','boathouse','carport','garages','bridge']:
        return 'transport' 
    elif x in ['industrial', 'manufacture','switchroom','storage']:
        return 'industrial'
    elif x in ['hospital', 'medical']:
        return 'medic'    
    elif x in ['ruins']:
        return 'ruins'    
    elif x in ['construction']:
        return 'construction_site'  
    else:
        return 'unknown'

gdf['build_program'] = gdf['building'].apply(lambda x: map_values(x))

In [89]:
gdf['building_l'].unique()

array([nan, '3', '6', '2', '19', '4', '5', '1', '11', '12', '25', '20',
       '9', '16', '7', '8', '22', '13', '10', '45', '47', '18', '14',
       '37', '27', '52', '23', '21', '30', '15', '41', '0', '17', '50',
       '26', '28', '33', '57', '55', '46', '54', '40', '43', '32', '1.5',
       '36', '31', '42', '24', '29', '-1', '2.0', '35', '38', '0.5',
       '12, 4', '4, 12', '34', '60', '63', '70', '39', '1,2,3', '65',
       '1+1', '48', '3.5', '1,2,3,4,5,6', '44', '56'], dtype=object)

In [125]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 113142 entries, 0 to 113141
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   osmid       113142 non-null  int32   
 1   addr_stree  49143 non-null   object  
 2   addr_house  53869 non-null   object  
 3   building_l  22530 non-null   object  
 4   building    113142 non-null  object  
 5   name        113142 non-null  object  
 6   build_prog  113142 non-null  object  
 7   geometry    113142 non-null  geometry
dtypes: geometry(1), int32(1), object(6)
memory usage: 6.5+ MB


In [102]:
#check if any column contains list or bytes - not permitted by shp!
for col in gdf.columns:
    if any(isinstance(val, bytes) for val in gdf[col]):
        print('Column: {0}, has a bytes in it'.format(col))
    if any(isinstance(val, list) for val in gdf[col]):
        print('Column: {0}, has a list in it'.format(col))

In [104]:
#retrieve exact names of rows with bytes datatype
import numpy as np
# create a boolean mask indicating rows containing byte strings
byte_mask = gdf['name'].apply(lambda x: isinstance(x, bytes))

# select rows containing byte strings
byte_rows = gdf[byte_mask]

# get unique byte string values
unique_bytes = np.unique(byte_rows['name'])
print(unique_bytes)

[]


In [105]:
gdf['name'] = gdf['name'].apply(lambda x: x.decode('latin-1') if isinstance(x, bytes) else x)

In [110]:
gdf = gdf.drop(columns=['addr_city','fid','office'])

In [120]:
gdf = gdf.rename(columns={'osm_id': 'osmid'})

In [121]:
gdf['osmid'] = gdf['osmid'].dropna().astype(int)

In [76]:
import pandas as pd

gdf1 = gdf.copy()
gdf1 = gdf1.apply(pd.to_numeric, errors='ignore')
gdf1 = gpd.GeoDataFrame(gdf1) # optional
gdf1.set_geometry(col='geometry', inplace=True)

In [124]:
# Save the modified shapefile
gdf.to_file('C:/WORK_IAAC/submissions/Internet of Buildings/IoB_WIP/IoB_QGIS/buildings/buildings_FIN.shp')